# Sign language recognition using deep learning models

In [ ]:
using Flux, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON, LinearAlgebra
using CSV
using Images
using GMT
using StatsBase

In [ ]:
#Reading the data
sign_mnist_test=CSV.read("sign_mnist_test.csv")
sign_mnist_train=CSV.read("sign_mnist_train.csv");

In [ ]:
#Counting the occurrences of individual letters in training and test sets
countmap(sign_mnist_test[:label])
countmap(sign_mnist_train[:label]);

In [ ]:
y_test=sign_mnist_test[:,1]
y_train=sign_mnist_train[:,1];

In [ ]:
x_test=sign_mnist_test[:,2:785]
x_train = sign_mnist_train[:,2:785];

In [ ]:
#Converting colors to grayscale
x_test=x_test./255
x_train=x_train./255;

In [ ]:
#Converting array [27455:784] to array [27455:[28:28]]
x_train1 = [reshape(Array{Float64}(x_train)[i,:], (28, 28)) for i in 1:size(Array{Float64}(x_train))[1]]
Gray.(x_train1[1])

In [ ]:
x_test1 = [reshape(Array{Float64}(x_test)[i,:], (28, 28)) for i in 1:size(Array{Float64}(x_test))[1]];

# Model 1

Convolutional neural network model

In [ ]:
#Function that partitions the set into batch_size partitions
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:24)
    return (X_batch, Y_batch)
end

In [ ]:
epochs = 100;
batch_size = 1000;

In [ ]:
#Partitioning the set
mb_idxs = partition(1:length(x_train1), batch_size)
train_set = [make_minibatch(x_train1, y_train, i) for i in mb_idxs];

In [ ]:
#Model definition
model = Chain(
    Conv((5, 5), 1=>6, stride=(1,1), relu),
    MaxPool((2,2)),
    Conv((5, 5), 6=>16, stride=(1,1), relu),
    MaxPool((2,2)),
    Conv((5, 5), 16=>200, pad=(1,1), relu),
    x -> reshape(x, :, size(x, 4)),
    Dense(800, 25),
    softmax,
);

#Loss function
function loss(x, y)
    x_aug = x .+ 0.1f0*randn(eltype(x), size(x))
    y_hat = model(x_aug)
    return crossentropy(y_hat, y)
end

accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))
opt = ADAM();

In [ ]:
test_set = make_minibatch(x_test1, y_test, 1:length(x_test1));

In [ ]:
#Training the model
@info("Beginning training loop...")
best_acc = 0.0
last_improvement = 0
for epoch = 1:epochs
    global best_acc, last_improvement
    Flux.train!(loss, params(model), train_set, opt)
    acc = accuracy(test_set[1],test_set[2])
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch, acc))
    if acc >= 0.90
        @info(" -> Early-exiting: We reached our target accuracy of 90%")
        break
    end
    if acc <= 0.05
        @info(" -> Best accurancy is: $(best_acc)")
        break
    end
    if acc >= best_acc
        @info(" -> New best accuracy! Saving model out to sign_lang_MNIST_conv.bson")
        BSON.@save "sign_lang_MNIST_conv.bson" model epoch acc
        best_acc = acc
        last_improvement = epoch
    end
    if epoch - last_improvement >= 5 && opt.eta > 1e-6
        opt.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")
        last_improvement = epoch
    end
    if epoch - last_improvement >= 10
        @warn(" -> We're calling this converged.")
        break
    end
end


# Model 2 

Multilayer perceptron model

In [ ]:
#Function that partitions the set into batch_size partitions
function make_minibatch(X, Y, idxs)
    X_batch = Array{Float32}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = Float32.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:24)
    return (X_batch, Y_batch)
end

In [ ]:
epochs = 500;
batch_size = 128;

In [ ]:
#Partitioning the set
mb_idxs = partition(1:length(x_train1), batch_size)
train_set = [make_minibatch(x_train1, y_train, i) for i in mb_idxs];

In [ ]:
test_set = make_minibatch(x_test1, y_test, 1:length(x_test1));

In [ ]:
#Model definition
mlp =   Chain(x -> reshape(x, :, size(x, 4)),
        Dense(784,256, relu),
        Dropout(0.4),
        Dense(256,128, relu),
        Dropout(0.4),
        Dense(128,25, relu),
        softmax,
)

α = 2.0f-6

#Loss function
function mlp_loss(x, y)
    y_hat = mlp(x)
    return crossentropy(y_hat, y) 
end
mlp_accuracy(x, y) = mean(onecold(mlp(x)) .== onecold(y))

opt = ADAM(0.0001)

In [ ]:
#Training the model
@info("Beginning training loop...")
best_acc = 0.0
last_improvement = 0
for epoch = 1:epochs
    global best_acc, last_improvement
    Flux.train!(mlp_loss, params(mlp), train_set, opt)
    acc = mlp_accuracy(test_set[1],test_set[2])
    @info(@sprintf("[%d]: Test accuracy: %.4f", epoch, acc))
    if acc >= 0.91
        @info(" -> Early-exiting: We reached our target accuracy of 90%")
        break
    end
    if acc >= best_acc
        @info(" -> New best accuracy! Saving model out to fashionMNIST_mlp.bson")
        BSON.@save "fashionMNIST_mlp.bson" mlp epoch acc
        best_acc = acc
        last_improvement = epoch
    end
    if epoch - last_improvement >= 5 && opt.eta > 1e-20
        opt.eta /= 10.0
        @warn(" -> Haven't improved in a while, dropping learning rate to $(opt.eta)!")
        last_improvement = epoch
    end
    if epoch - last_improvement >= 10
        @warn(" -> We're calling this converged. Best acc: $(best_acc)")
        break
    end
end